In [6]:
# Source https://towardsdatascience.com/looking-for-a-house-build-a-web-scraper-to-help-you-5ab25badc83e
# import libraries
from urllib2 import urlopen
from requests import get
from bs4 import BeautifulSoup
from time import sleep
from random import randint
import time
import re


import pandas as pd
import numpy as np
import pylab as pl

from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
import warnings

In [7]:
url = 'https://raw.githubusercontent.com/jems2019/CMPE257/master/data/wine_fire_weather_data.csv'
data = pd.read_csv(url)


In [14]:
data.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,...,norm_price,norm_points,norm_total_fires,norm_sum_fire_size,norm_total_fires_current,norm_sum_fire_size_current,mean_temp,mean_rain,norm_temp,norm_rain
0,0,US,"Soft, supple plum envelopes an oaky structure ...",Mountain Cuvée,87,19.0,California,Napa Valley,Napa,Virginie Boone,...,0.006955,0.070707,0.000000,0.000000,0.450893,0.006877,59.757174,18.040338,0.219366,0.137560
1,1,US,This wine from the Geneseo district offers aro...,Signature Selection,87,22.0,California,Paso Robles,Central Coast,Matt Kettmann,...,0.008445,0.070707,0.008929,0.000310,0.428571,0.002502,57.042925,23.084964,0.176770,0.192200
2,2,US,Oak and earth intermingle around robust aromas...,King Ridge Vineyard,87,69.0,California,Sonoma Coast,Sonoma,Virginie Boone,...,0.031793,0.070707,0.000000,0.000000,0.450893,0.006877,59.757174,18.040338,0.219366,0.137560
3,3,US,"Syrupy and dense, this wine is jammy in plum a...",Estate,86,100.0,California,Napa Valley,Napa,Virginie Boone,...,0.047193,0.060606,0.272321,0.005246,0.551339,0.005689,61.338887,5.535779,0.244188,0.002119
4,4,US,There are intriguing touches to the nose of th...,Golden Horn,86,26.0,California,Santa Ynez Valley,Central Coast,Matt Kettmann,...,0.010432,0.060606,0.183036,0.002815,0.243304,0.010616,60.220294,12.206211,0.226633,0.074369


In [0]:
# Load the cleaned data into dataframe

""" 
    Information of cleaned dataset:
    
    - Province and Variety of wine is transfomed from a categorical attribute to numerical attribute
    - Vintage of wine is extracted from the title of each wine
    - Rows were removed if there were null values present within each of the three attributes (Province, Variety, Vintage)
    
"""
warnings.simplefilter('ignore')
#data = pd.read_csv("../California_Wines.csv")
data.dropna(inplace = True)
print(data.shape)
data.head()

(12638, 18)


,Unnamed: 0,Unnamed: 0.1,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,province_id,variety_numerical,vintage
0,10,10,US,"Soft, supple plum envelopes an oaky structure ...",Mountain Cuvée,87,19.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Kirkland Signature 2011 Mountain Cuvée Caberne...,Cabernet Sauvignon,Kirkland Signature,7,80,2011
3,23,23,US,This wine from the Geneseo district offers aro...,Signature Selection,87,22.0,California,Paso Robles,Central Coast,Matt Kettmann,@mattkettmann,Bianchi 2011 Signature Selection Merlot (Paso ...,Merlot,Bianchi,7,326,2011
4,25,25,US,Oak and earth intermingle around robust aromas...,King Ridge Vineyard,87,69.0,California,Sonoma Coast,Sonoma,Virginie Boone,@vboone,Castello di Amorosa 2011 King Ridge Vineyard P...,Pinot Noir,Castello di Amorosa,7,440,2011
12,60,60,US,"Syrupy and dense, this wine is jammy in plum a...",Estate,86,100.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Okapi 2013 Estate Cabernet Sauvignon (Napa Val...,Cabernet Sauvignon,Okapi,7,80,2013
13,63,64,US,There are intriguing touches to the nose of th...,Golden Horn,86,26.0,California,Santa Ynez Valley,Central Coast,Matt Kettmann,@mattkettmann,Sevtap 2015 Golden Horn Sauvignon Blanc (Santa...,Sauvignon Blanc,Sevtap,7,514,2015
15,69,71,US,Big oak defines this robustly dense and extrac...,Old Vine,86,40.0,California,Alexander Valley,Sonoma,Virginie Boone,@vboone,Eco Terreno 2013 Old Vine Cabernet Sauvignon (...,Cabernet Sauvignon,Eco Terreno,7,80,2013
16,71,73,US,"Juicy plum, raspberry and pencil lead lead the...",Bella Vetta Vineyard,86,75.0,California,Howell Mountain,Napa,Virginie Boone,@vboone,Hindsight 2013 Bella Vetta Vineyard Cabernet S...,Cabernet Sauvignon,Hindsight,7,80,2013
17,72,74,US,This is the producer's first release of this w...,Estate Grown,86,55.0,California,Calistoga,Napa,Virginie Boone,@vboone,Hindsight 2012 Estate Grown Petite Sirah (Cali...,Petite Sirah,Hindsight,7,419,2012
18,73,75,US,This is made from equal parts Cabernet Franc a...,The Cypher,86,75.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Mulvane Wine Co. 2013 The Cypher Red (Napa Val...,Bordeaux-style Red Blend,Mulvane Wine Co.,7,62,2013
19,82,84,US,A healthy addition of 13% Petite Sirah provide...,Classic,86,24.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Napa Cellars 2014 Classic Zinfandel (Napa Valley),Zinfandel,Napa Cellars,7,702,2014


In [0]:
d1= data.head(10)
d1

,Unnamed: 0,Unnamed: 0.1,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,province_id,variety_numerical,vintage
0,10,10,US,"Soft, supple plum envelopes an oaky structure ...",Mountain Cuvée,87,19.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Kirkland Signature 2011 Mountain Cuvée Caberne...,Cabernet Sauvignon,Kirkland Signature,7,80,2011
3,23,23,US,This wine from the Geneseo district offers aro...,Signature Selection,87,22.0,California,Paso Robles,Central Coast,Matt Kettmann,@mattkettmann,Bianchi 2011 Signature Selection Merlot (Paso ...,Merlot,Bianchi,7,326,2011
4,25,25,US,Oak and earth intermingle around robust aromas...,King Ridge Vineyard,87,69.0,California,Sonoma Coast,Sonoma,Virginie Boone,@vboone,Castello di Amorosa 2011 King Ridge Vineyard P...,Pinot Noir,Castello di Amorosa,7,440,2011
12,60,60,US,"Syrupy and dense, this wine is jammy in plum a...",Estate,86,100.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Okapi 2013 Estate Cabernet Sauvignon (Napa Val...,Cabernet Sauvignon,Okapi,7,80,2013
13,63,64,US,There are intriguing touches to the nose of th...,Golden Horn,86,26.0,California,Santa Ynez Valley,Central Coast,Matt Kettmann,@mattkettmann,Sevtap 2015 Golden Horn Sauvignon Blanc (Santa...,Sauvignon Blanc,Sevtap,7,514,2015
15,69,71,US,Big oak defines this robustly dense and extrac...,Old Vine,86,40.0,California,Alexander Valley,Sonoma,Virginie Boone,@vboone,Eco Terreno 2013 Old Vine Cabernet Sauvignon (...,Cabernet Sauvignon,Eco Terreno,7,80,2013
16,71,73,US,"Juicy plum, raspberry and pencil lead lead the...",Bella Vetta Vineyard,86,75.0,California,Howell Mountain,Napa,Virginie Boone,@vboone,Hindsight 2013 Bella Vetta Vineyard Cabernet S...,Cabernet Sauvignon,Hindsight,7,80,2013
17,72,74,US,This is the producer's first release of this w...,Estate Grown,86,55.0,California,Calistoga,Napa,Virginie Boone,@vboone,Hindsight 2012 Estate Grown Petite Sirah (Cali...,Petite Sirah,Hindsight,7,419,2012
18,73,75,US,This is made from equal parts Cabernet Franc a...,The Cypher,86,75.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Mulvane Wine Co. 2013 The Cypher Red (Napa Val...,Bordeaux-style Red Blend,Mulvane Wine Co.,7,62,2013
19,82,84,US,A healthy addition of 13% Petite Sirah provide...,Classic,86,24.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Napa Cellars 2014 Classic Zinfandel (Napa Valley),Zinfandel,Napa Cellars,7,702,2014


In [0]:
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})

In [0]:
# specify the url
quote_page = 'https://www.yelp.com/search?find_desc=winery&find_loc=Napa%2C%20CA'

In [0]:
response = get(quote_page, headers=headers)
print(response)

<Response [200]>


In [0]:
#print(response.text)

In [0]:
# parse the html using beautiful soup and store in variable `soup`
soup = BeautifulSoup(response.text, 'html.parser')

In [0]:
# Take out the <div> of name and get its value
name_box = soup.find('a', attrs={'class': 'lemon--a__373c0__IEZFH link__373c0__29943 link-color--blue-dark__373c0__1mhJo link-size--inherit__373c0__2JXk5'})


In [0]:
########## WEATHER 
# specify the url
sample = 'https://www.wunderground.com/history/monthly/us/ca/santa-rosa/KSTS/date/1990-3'


chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
wd.get(sample)

sample_soup = BeautifulSoup(wd.page_source, 'html.parser')
sample_soup.text
#print (wd.page_source.encode('utf-8'))

#python_button = wd.find_element_by_id('summary-table')
#python_button.click() #click fhsu link



#sample_response = get(sample, headers=headers)
#print(sample_response)
#print(sample_response.text)
#sample_soup = BeautifulSoup(sample_response.text, 'html.parser')
#sample_name_box = sample_soup.find('div', attrs={'class': 'summary-table'}).table
#print(sample_name_box)


'\nSanta Rosa, CA History | Weather Underground\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n  //&lt;![CDATA[\n  window.webpackManifest = {"0":"city-history-module.9af2efef01a92b6753b4.js","1":"video-module.4338954dc3409bd5950a.js","2":"health-module.8f697267290f6743a285.js","3":"city-today-module.f036d0ff5e70b05214b5.js","4":"hurricane-module.6e3d18105c06bf3d60ee.js","5":"city-ten-day-module.27afc070f47a91d1fb94.js","6":"city-hourly-module.08c1b5d585f564a32d37.js","7":"precipitation-module.96aa4cd20eb24bc4767c.js","8":"city-history-calendar-module.0e6eb964d7c1d7a1e5fe.js","9":"city-severe-module.59ea8818e3884f3050ce.js","10":"radar-module.c278a90dcf42f203b474.js","11":"article-page-module.ce44f302c19f86d9be86.js","12":"page-module.3504dd1f2002f2ce48a5.js","13":"member-mydevices-module.b26a20c0e9b434f78bac.js","14":"landing-purpleair-module.c803a4da2f289e5d5063.js","15":"test-module.c0f7f86b6833aa6fb2f5.js","16":"hurricane-storm-module.b39a6557df69025a4175.js","17":"w

In [0]:
name = name_box.text.strip() # strip() is used to remove starting and trailing
print(name)

Patz & Hall Winery


In [0]:
# Get Yelp search result container

yelp_containers = soup.find_all('div', class_="lemon--div__373c0__1mboc u-padding-t3 u-padding-b3 border-color--default__373c0__2oFDT")

In [0]:
# Get the first winery result
first = yelp_containers[0]

# Get the winery name
name = first.find('a', attrs={'class': 'lemon--a__373c0__IEZFH link__373c0__29943 link-color--blue-dark__373c0__1mhJo link-size--inherit__373c0__2JXk5'})
name.text

'Patz & Hall Winery'

In [0]:
# Get the winery rating
rating = first.find('span', attrs={'class': 'lemon--span__373c0__3997G display--inline__373c0__1DbOG border-color--default__373c0__2oFDT'}).div.get("aria-label")
rating

'4.5 star rating'

In [0]:
d10= data.head(10)

# get the winery name for the dataframe
d10.iloc[0,d10.columns.get_loc("winery")]
d10

,Unnamed: 0,Unnamed: 0.1,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,province_id,variety_numerical,vintage
0,10,10,US,"Soft, supple plum envelopes an oaky structure ...",Mountain Cuvée,87,19.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Kirkland Signature 2011 Mountain Cuvée Caberne...,Cabernet Sauvignon,Kirkland Signature,7,80,2011
3,23,23,US,This wine from the Geneseo district offers aro...,Signature Selection,87,22.0,California,Paso Robles,Central Coast,Matt Kettmann,@mattkettmann,Bianchi 2011 Signature Selection Merlot (Paso ...,Merlot,Bianchi,7,326,2011
4,25,25,US,Oak and earth intermingle around robust aromas...,King Ridge Vineyard,87,69.0,California,Sonoma Coast,Sonoma,Virginie Boone,@vboone,Castello di Amorosa 2011 King Ridge Vineyard P...,Pinot Noir,Castello di Amorosa,7,440,2011
12,60,60,US,"Syrupy and dense, this wine is jammy in plum a...",Estate,86,100.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Okapi 2013 Estate Cabernet Sauvignon (Napa Val...,Cabernet Sauvignon,Okapi,7,80,2013
13,63,64,US,There are intriguing touches to the nose of th...,Golden Horn,86,26.0,California,Santa Ynez Valley,Central Coast,Matt Kettmann,@mattkettmann,Sevtap 2015 Golden Horn Sauvignon Blanc (Santa...,Sauvignon Blanc,Sevtap,7,514,2015
15,69,71,US,Big oak defines this robustly dense and extrac...,Old Vine,86,40.0,California,Alexander Valley,Sonoma,Virginie Boone,@vboone,Eco Terreno 2013 Old Vine Cabernet Sauvignon (...,Cabernet Sauvignon,Eco Terreno,7,80,2013
16,71,73,US,"Juicy plum, raspberry and pencil lead lead the...",Bella Vetta Vineyard,86,75.0,California,Howell Mountain,Napa,Virginie Boone,@vboone,Hindsight 2013 Bella Vetta Vineyard Cabernet S...,Cabernet Sauvignon,Hindsight,7,80,2013
17,72,74,US,This is the producer's first release of this w...,Estate Grown,86,55.0,California,Calistoga,Napa,Virginie Boone,@vboone,Hindsight 2012 Estate Grown Petite Sirah (Cali...,Petite Sirah,Hindsight,7,419,2012
18,73,75,US,This is made from equal parts Cabernet Franc a...,The Cypher,86,75.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Mulvane Wine Co. 2013 The Cypher Red (Napa Val...,Bordeaux-style Red Blend,Mulvane Wine Co.,7,62,2013
19,82,84,US,A healthy addition of 13% Petite Sirah provide...,Classic,86,24.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Napa Cellars 2014 Classic Zinfandel (Napa Valley),Zinfandel,Napa Cellars,7,702,2014


In [224]:
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at drive


In [49]:
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})


yelp_url = 'https://www.yelp.com/search?find_desc='


#d10= data.head(3)
start = 0
end = unique_wineries.shape[0]


# setting up the lists that will form the dataframe with all the winery ratings
winery_list = []
ratings_list = []
num_reviews = []
dollar_list = []

  
cols = ['Winery', 'Ratings', 'Number of Reviews', 'Yelp Dollar Signs']

res = pd.DataFrame({'Winery': winery_list, 
                   'Ratings': ratings_list,
                   'Number of Reviews': num_reviews,
                   'Yelp Dollar Signs': dollar_list})[cols]

start_time = time.time()
finished = 0

while (start < end):
    winery = unique_wineries.iloc[start,unique_wineries.columns.get_loc("winery")]
    location = unique_wineries.iloc[start,unique_wineries.columns.get_loc("region_1")]
    url = yelp_url + winery + '&find_loc=' + location + ', CA'
    response = get(url, headers=headers)
      
    print(url)

    #print(response)
  
    # parse the html using beautiful soup and store in variable `soup`
    soup = BeautifulSoup(response.text, 'html.parser')
    
    #print(soup)
  
    winery_containers = soup.find_all('div', class_="lemon--div__373c0__1mboc searchResult__373c0__1yggB border-color--default__373c0__2oFDT")
    
    if winery_containers != []:
        i = 0
    
        for container in winery_containers:

          # get the winery name from Yelp
            soup_name = winery_containers[i].find_all('h3')[0].text

          # if the soup name contains the winery name 
          # and the soup name has winery or vineyard      
            if (str(winery) in str(soup_name)): # and ('winery' in soup_name.lower() or 'vineyard' in soup_name.lower()):
                winery_list.append(winery)

                # get the winery rating from Yelp
                soup_rating = winery_containers[i].find('span', attrs={'class': 'lemon--span__373c0__3997G display--inline__373c0__1DbOG border-color--default__373c0__2oFDT'}).div.get("aria-label")
                if (soup_rating):
                    ratings_list.append(float(soup_rating.replace(' star rating', '')))
                else:
                    ratings_list.append(float('NaN'))
                

                # get the number of Yelp reviews for the winery
                soup_review = winery_containers[i].find('span', attrs={'class': 'lemon--span__373c0__3997G text__373c0__2pB8f reviewCount__373c0__2r4xT text-color--mid__373c0__3G312 text-align--left__373c0__2pnx_'}).text
                if (soup_review):
                    num_reviews.append(soup_review.replace(' reviews', ''))
                else:
                    num_reviews.append(float('NaN'))


                # get the number of Yelp dollar signs for the winery
                soup_dollars = winery_containers[i].find('span', attrs={'class': 'lemon--span__373c0__3997G text__373c0__2pB8f priceRange__373c0__2DY87 text-color--normal__373c0__K_MKN text-align--left__373c0__2pnx_ text-bullet--after__373c0__1ZHaA'})
                if (soup_dollars):
                    dollar_list.append(soup_dollars.text.count('$'))
                else:
                    dollar_list.append(float('NaN'))      
      
        i += 1

   
    start+=1
  
    finished += 1
    
    #if (finished%10 == 0):
    print('finished' , finished, '/', unique_wineries.shape[0], ' --- time elapsed:', time.time() - start_time)
    
    sleep(randint(0,1))
  




#print(winery_list)
#print(ratings_list)
#print(num_reviews)
#print(dollar_list)

res.to_csv('data.csv')
#!cp data.csv drive/My\ Drive/



AttributeError: 'numpy.ndarray' object has no attribute 'iloc'

In [8]:
# gets all the unique wineries and region from the dataset
unique_wineries = data.drop_duplicates(subset=['region_1', 'winery'], keep='first', inplace=False)
unique_wineries.shape
#pd.set_option('display.max_columns', 50)
#unique_df.head()


unique_wineries.iloc[0,unique_wineries.columns.get_loc("winery")]
unique_wineries.to_csv("unique_wineries.csv")

In [9]:
unique_wineries = pd.read_csv("unique_wineries.csv")

In [10]:
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})


yelp_url = 'https://www.yelp.com/search?find_desc='


#d10= data.head(3)
start = 0
end = unique_wineries.shape[0]



# setting up the lists that will form the dataframe with all the winery ratings
winery_list = []
ratings_list = []
num_reviews = []
dollar_list = []

  
start_time = time.time()
finished = 0
partition = 1

while (start< end):
    winery = unique_wineries.iloc[start,unique_wineries.columns.get_loc("winery")]
    location = unique_wineries.iloc[start,unique_wineries.columns.get_loc("region_1")]
    url = yelp_url + winery + '&find_loc=' + location + ', CA'
    response = get(url, headers=headers)
    
    print(url)
  
    # parse the html using beautiful soup and store in variable `soup`
    soup = BeautifulSoup(response.text, 'html.parser')

    if ('robot' in soup.text):
        print('CAPTCHA')
    
    winery_containers = soup.find_all('div', class_="lemon--div__373c0__1mboc searchResult__373c0__1yggB border-color--default__373c0__2oFDT")
    i = 0
    if winery_containers != []:  
        for container in winery_containers:

          # get the winery name from Yelp
            soup_name = winery_containers[i].find_all('h3')[0].text

          # if the soup name contains the winery name 
          # and the soup name has winery or vineyard      
            if (str(winery) in str(soup_name)): # and ('winery' in soup_name.lower() or 'vineyard' in soup_name.lower()):
                winery_list.append(winery)

                
                 # get the winery rating from Yelp
                soup_rating = winery_containers[i].find('span', attrs={'class': 'lemon--span__373c0__3997G display--inline__373c0__1DbOG border-color--default__373c0__2oFDT'})
                if (soup_rating):
                    #print(soup_rating.get("aria-label"))
                    #match = str(re.search(r' star.\w+', soup_rating.get("aria-label")).group(0))
                    #soup_rating_val = soup_rating.get("aria-label").replace(match, '') 
                    rating = soup_rating.div.get("aria-label").replace(' star rating', '')
                    ratings_list.append(rating)
                else:
                    ratings_list.append(float('NaN'))
        
                # get the number of Yelp reviews for the winery
                soup_review = winery_containers[i].find('span', attrs={'class': 'lemon--span__373c0__3997G text__373c0__2pB8f reviewCount__373c0__2r4xT text-color--mid__373c0__3G312 text-align--left__373c0__2pnx_'})
                if (soup_review):
                    match = str(re.search(r' review\w*', soup_review.text).group(0))
                    soup_review_val = soup_review.text.replace(match, '')
                    num_reviews.append(soup_review_val)
                else:
                    num_reviews.append(float('NaN'))
        
        
                # get the number of Yelp dollar signs for the winery
                soup_dollars = winery_containers[i].find('span', attrs={'class': 'lemon--span__373c0__3997G text__373c0__2pB8f priceRange__373c0__2DY87 text-color--normal__373c0__K_MKN text-align--left__373c0__2pnx_ text-bullet--after__373c0__1ZHaA'})
                if (soup_dollars):
                    dollar_list.append(soup_dollars.text)
                else:
                    dollar_list.append(float('NaN'))
    
      
    
            i += 1

   
    start+=1 # counter
    finished += 1 # number of wineries completed
    
    print('finished' , finished, '/', unique_wineries.shape[0], ' --- time elapsed:', time.time() - start_time)
    
    sleep(randint(0,1))
    if (finished % 25 == 0):
        data_name = 'data_' + str(partition) + '.csv'

        cols = ['Winery', 'Ratings', 'Number of Reviews', 'Yelp Dollar Signs']
        res = pd.DataFrame({'Winery': winery_list, 'Ratings': ratings_list, 'Number of Reviews': num_reviews, 'Yelp Dollar Signs': dollar_list})[cols]
        res.to_csv(data_name)
        
        partition += 1
    
    
'''
cols = ['Winery', 'Ratings', 'Number of Reviews', 'Yelp Dollar Signs']

res = pd.DataFrame({'Winery': winery_list, 
                   'Ratings': ratings_list,
                   'Number of Reviews': num_reviews,
                   'Yelp Dollar Signs': dollar_list})[cols]

print(winery_list)
print(ratings_list)
print(num_reviews)
print(dollar_list)

#res.to_csv('data.csv')
#!cp data.csv drive/My\ Drive/
'''



https://www.yelp.com/search?find_desc=Kirkland Signature&find_loc=Napa Valley, CA


UnicodeEncodeError: 'ascii' codec can't encode character u'\xa0' in position 2: ordinal not in range(128)

In [ ]:
    if (finished % 1 == 0):
        data_name = 'data_' + str(partition) + '.csv'
        print(winery_list)
        print(ratings_list)
        print(num_reviews)
        print(dollar_list)
        cols = ['Winery', 'Ratings', 'Number of Reviews', 'Yelp Dollar Signs']
        res = pd.DataFrame({'Winery': winery_list, 'Ratings': ratings_list, 'Number of Reviews': num_reviews, 'Yelp Dollar Signs': dollar_list})[cols]
        res.to_csv(data_name)
        
        partition += 1

In [64]:
match = str(re.search(r' g\w+', '5 gssss').group(0))
print(match)
'5 gssss'.replace(match, '')

 gssss


'5'

In [75]:
match = str(re.search(r' review\w*', '1 review').group(0))
match

' review'